In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_out=pd.read_csv("/content/drive/MyDrive/train_out.csv")
train_out.drop(["Unnamed: 0"], axis=1, inplace=True)
train_out.head(2)

,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
0,2,1,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.463330,34.160000,53.996091,0,0.000000,0.20000,0.0,0.041667,0.025000,0.0,0.000000,0.175,0.033333,0.0,1
1,2,2,0.0,0.0,0.0,0,0.0,0.0,0.076923,0.153846,0.429468,15.285714,96.231625,0,0.008475,0.09322,0.0,0.012712,0.018644,0.0,0.084746,0.000,0.033898,82.5,2


In [4]:
train_out.columns

Index(['tier', 'gender', 'following_rate', 'followers_avg_age',
       'following_avg_age', 'max_repetitive_punc',
       'num_of_hashtags_per_action', 'emoji_count_per_action',
       'punctuations_per_action', 'number_of_words_per_action',
       'avgCompletion', 'avgTimeSpent', 'avgDuration', 'avgComments',
       'creations', 'content_views', 'num_of_comments',
       'weekends_trails_watched_per_day', 'weekdays_trails_watched_per_day',
       'slot1_trails_watched_per_day', 'slot2_trails_watched_per_day',
       'slot3_trails_watched_per_day', 'slot4_trails_watched_per_day', 'avgt2',
       'age_group'],
      dtype='object')

In [5]:
y = train_out['age_group']
X = train_out.drop(['age_group'], axis=1, inplace=False)

In [ ]:
from imblearn.combine import SMOTETomek
smk = SMOTETomek(random_state=42)
X_res,y_res=smk.fit_sample(X,y)

In [24]:
data_X = pd.DataFrame(X_res, columns = ['tier', 'gender', 'following_rate', 'followers_avg_age',
       'following_avg_age', 'max_repetitive_punc',
       'num_of_hashtags_per_action', 'emoji_count_per_action',
       'punctuations_per_action', 'number_of_words_per_action',
       'avgCompletion', 'avgTimeSpent', 'avgDuration', 'avgComments',
       'creations', 'content_views', 'num_of_comments',
       'weekends_trails_watched_per_day', 'weekdays_trails_watched_per_day',
       'slot1_trails_watched_per_day', 'slot2_trails_watched_per_day',
       'slot3_trails_watched_per_day', 'slot4_trails_watched_per_day', 'avgt2'])
data_X.head(2)

,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2
0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.463330,34.160000,53.996091,0.0,0.000000,0.20000,0.0,0.041667,0.025000,0.0,0.000000,0.175,0.033333,0.0
1,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.153846,0.429468,15.285714,96.231625,0.0,0.008475,0.09322,0.0,0.012712,0.018644,0.0,0.084746,0.000,0.033898,82.5


In [26]:
data_X.shape

(1215750, 24)

In [7]:
train_x,test_x,train_y,test_y=train_test_split(X_res,y_res,test_size=.04,random_state=42, stratify = y_res)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train=scaler.fit_transform(train_x)
x_test=scaler.transform(test_x)

In [9]:
xgb_clf = XGBClassifier(tree_method = 'gpu_hist', gpu_id=0, objective = "multi:softmax", num_class=4,n_estimators=1500, max_depth=20,learning_rate=0.006)

In [10]:
xgb_hist = xgb_clf.fit(x_train, train_y)

In [11]:
from sklearn import metrics
expected_y  = test_y
predicted_y = xgb_clf.predict(x_test)
print('Report: ') 
print(metrics.classification_report(expected_y, predicted_y))

Report: 
              precision    recall  f1-score   support

           1       0.95      0.88      0.91     12097
           2       0.76      0.85      0.80     12175
           3       0.81      0.77      0.79     12175
           4       0.80      0.80      0.80     12183

    accuracy                           0.82     48630
   macro avg       0.83      0.82      0.82     48630
weighted avg       0.83      0.82      0.82     48630



In [14]:
test=pd.read_csv('/content/drive/MyDrive/test_age_dataset.csv')
test.drop(["Unnamed: 0","userId"], axis=1, inplace=True)
test.head(2)

,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2
0,2,1,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0,0.033242,4.5,120.313571,0,0.0,0.062992,0.0,0.0,0.012598,0.0,0.0,0.062992,0.0,0.0
1,2,1,0.001818,0.0,0.0,0,0.0,0.0,0.0,0.0,0.051057,9.5,188.615500,0,0.0,0.003636,0.0,0.0,0.000727,0.0,0.0,0.003636,0.0,0.0


In [15]:
test.values

array([[2.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        6.29921260e-02, 0.00000000e+00, 0.00000000e+00],
       [2.00000000e+00, 1.00000000e+00, 1.81818200e-03, ...,
        3.63636400e-03, 0.00000000e+00, 0.00000000e+00],
       [2.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        7.42857143e-01, 3.14285714e-01, 0.00000000e+00],
       ...,
       [2.00000000e+00, 1.00000000e+00, 4.83870970e-02, ...,
        4.83870970e-02, 7.25806452e-01, 1.86000000e+02],
       [1.00000000e+00, 2.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.77966102e-01, 0.00000000e+00],
       [3.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [16]:
predict = xgb_clf.predict(test.values)

In [17]:
df_pred = pd.DataFrame({"prediction" : predict}, columns = ["prediction"]).set_index("prediction")

In [18]:
def download_preds(df_pred, file_name = 'res3.csv'):
  df_pred.to_csv(file_name)
  from google.colab import files
  files.download(file_name)
download_preds(df_pred, file_name = 'res3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>